In [2]:
%matplotlib inline


Neural Networks
===============

Neural networks can be constructed using the ``torch.nn`` package.

Now that you had a glimpse of ``autograd``, ``nn`` depends on
``autograd`` to define models and differentiate them.
An ``nn.Module`` contains layers, and a method ``forward(input)``\ that
returns the ``output``.

For example, look at this network that classifies digit images:

.. figure:: /_static/img/mnist.png
   :alt: convnet

   convnet

It is a simple feed-forward network. It takes the input, feeds it
through several layers one after the other, and then finally gives the
output.

A typical training procedure for a neural network is as follows:

- Define the neural network that has some learnable parameters (or
  weights)
- Iterate over a dataset of inputs
- Process input through the network
- Compute the loss (how far is the output from being correct)
- Propagate gradients back into the network’s parameters
- Update the weights of the network, typically using a simple update rule:
  ``weight = weight - learning_rate * gradient``

Define the network
------------------

Let’s define this network:



first create a function to check grad_gn

In [3]:
# print all attributes
def DCG_node(listValue):
    for (i, v) in enumerate(listValue):
        print(f"{i}.data: {v.data}\n{i}.requires_grad: {v.requires_grad}\n{i}.grad: {v.grad}\n{i}.grad_fn: {v.grad_fn}\n{i}.is_leaf: {v.is_leaf}\n")

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

Conv layer
-----------------------
we create a image = $N$, $T$, $x$, $y$ represents batch size, input_channel, img_height, img_width  
This time we generate a simple one, just two of 6 * 6 grey image whose input channel is one

In [17]:
img = torch.FloatTensor([[[[1, 1, 1, 1, 1, 1],
                           [0, 0, 0, 0, 0, 0],
                           [1, 1, 1, 1, 1, 1],
                           [0, 0, 0, 0, 0, 0],
                           [1, 1, 1, 1, 1, 1],
                           [0, 0, 0, 0, 0, 0]]], 
                         
                         [[[0, 1, 0, 1, 0, 1],
                           [0, 1, 0, 1, 0, 1],
                           [0, 1, 0, 1, 0, 1],
                           [0, 1, 0, 1, 0, 1],
                           [0, 1, 0, 1, 0, 1],
                           [0, 1, 0, 1, 0, 1]]]])
img.size()

torch.Size([2, 1, 6, 6])

put it in the convolutional layer first, at this time kernel just generate randomly since we need to try them

In [21]:
conv = nn.Conv2d(1, 1, 3) # the input channel should be consistent with the image
conv_img = conv(img)
conv_img

tensor([[[[ 0.1485,  0.1485,  0.1485,  0.1485],
          [-0.1226, -0.1226, -0.1226, -0.1226],
          [ 0.1485,  0.1485,  0.1485,  0.1485],
          [-0.1226, -0.1226, -0.1226, -0.1226]]],


        [[[-0.0807,  0.1067, -0.0807,  0.1067],
          [-0.0807,  0.1067, -0.0807,  0.1067],
          [-0.0807,  0.1067, -0.0807,  0.1067],
          [-0.0807,  0.1067, -0.0807,  0.1067]]]],
       grad_fn=<MkldnnConvolutionBackward>)

check the paramaters of this conv layer

In [22]:
params = list(conv.parameters())
params

[Parameter containing:
 tensor([[[[-0.1112,  0.2621,  0.0453],
           [ 0.0957, -0.2760, -0.0305],
           [ 0.0161, -0.1551,  0.0031]]]], requires_grad=True),
 Parameter containing:
 tensor([0.0882], requires_grad=True)]

they kernel layer(weights) cover on the image to calculate (multiple and then summary) one scalar and then add bias, so all layers in one kernel cube add the same scalar. The number of bias is the same as the number of kernel cube.

ReLU
--------------------------------
ReLU it convert negative to 0

In [23]:
relu_img = F.relu(conv_img)
relu_img

tensor([[[[0.1485, 0.1485, 0.1485, 0.1485],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.1485, 0.1485, 0.1485, 0.1485],
          [0.0000, 0.0000, 0.0000, 0.0000]]],


        [[[0.0000, 0.1067, 0.0000, 0.1067],
          [0.0000, 0.1067, 0.0000, 0.1067],
          [0.0000, 0.1067, 0.0000, 0.1067],
          [0.0000, 0.1067, 0.0000, 0.1067]]]], grad_fn=<ReluBackward0>)

max_pooling
-------------------------
max_pooling it

In [24]:
mxp_img = F.max_pool2d(relu_img, (2, 2))
print(mxp_img)
mxp_img.size() # batch size, input channel, img_height and width

tensor([[[[0.1485, 0.1485],
          [0.1485, 0.1485]]],


        [[[0.1067, 0.1067],
          [0.1067, 0.1067]]]], grad_fn=<MaxPool2DWithIndicesBackward>)


torch.Size([2, 1, 2, 2])

In [25]:
size = mxp_img.size()[1:]  # all dimensions except the batch dimension
num_features = 1
for s in size:
    num_features *= s
print(num_features) # 1 * 4 * 4 = 16

4


linear layer
-------------------------
before put it into the linear layer, we need to view this mxp_img

In [27]:
mxp_img = mxp_img.view(-1, 4)
mxp_img

tensor([[0.1485, 0.1485, 0.1485, 0.1485],
        [0.1067, 0.1067, 0.1067, 0.1067]], grad_fn=<ViewBackward>)

add one linear layer

In [30]:
fc = nn.Linear(4, 10) # input is 16 and output is 30
fc_img = fc(mxp_img)

check the parameter

In [31]:
params = list(fc.parameters())
params

[Parameter containing:
 tensor([[-0.1685, -0.3277, -0.0822, -0.3385],
         [-0.3645, -0.3390,  0.0490,  0.0228],
         [-0.1691, -0.3650, -0.2402,  0.1799],
         [-0.2702, -0.1546, -0.0353,  0.0486],
         [ 0.0428, -0.0830, -0.4213,  0.3464],
         [-0.2968, -0.1012, -0.3677,  0.2449],
         [ 0.0873, -0.1390,  0.2914, -0.4804],
         [-0.1675, -0.1392,  0.0906, -0.0490],
         [-0.3770, -0.0596, -0.4381,  0.1597],
         [-0.2175, -0.1880, -0.0034,  0.3303]], requires_grad=True),
 Parameter containing:
 tensor([ 0.0566, -0.1509, -0.4600, -0.2425, -0.1440,  0.0772,  0.4097, -0.0398,
          0.1655, -0.0527], requires_grad=True)]

check the image

In [32]:
fc_img

tensor([[-7.9556e-02, -2.4475e-01, -5.4829e-01, -3.0359e-01, -1.6108e-01,
         -1.6225e-04,  3.7390e-01, -7.9219e-02,  5.9309e-02, -6.4365e-02],
        [-4.1197e-02, -2.1832e-01, -5.2342e-01, -2.8637e-01, -1.5626e-01,
          2.1627e-02,  3.8397e-01, -6.8129e-02,  8.9224e-02, -6.1074e-02]],
       grad_fn=<AddmmBackward>)

so you can see, the process is that
```
original image (batch size: *, in_channel: IN, height: *, width: *)-> conv layer (in_channel: IN, out_channel: OUT, kernel_size: S) -> ReLU layer -> max pooling -> linear layer
```
not matter what size of image you would like to use, but just the input channel of the image will be affected  
**So the model just care about the input channel of these images.**

neural network architecture
----------------------------------

In [44]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels means 6 kernel cube
        # with 1 layer 3x3 square 
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension 
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


You just have to define the ``forward`` function, and the ``backward``
function (where gradients are computed) is automatically defined for you
using ``autograd``.
You can use any of the Tensor operations in the ``forward`` function.

The learnable parameters of a model are returned by ``net.parameters()``



In [46]:
params = list(net.parameters())
print(len(params))
for p in params:
    print(p.size())
    print(' ')

# kernel cube weigths: number of kernel cube, layer of each kernel cube, shape of kernel layer is 3 * 3
# torch.Size([6, 1, 3, 3])
 
# kernel cube bias: for each kernel cube
# torch.Size([6])
 
# torch.Size([16, 6, 3, 3])
 
# torch.Size([16])
 
# torch.Size([120, 576])
 
# torch.Size([120])
 
# torch.Size([84, 120])
 
# torch.Size([84])
 
# torch.Size([10, 84])
 
# torch.Size([10])

10
torch.Size([6, 1, 3, 3])
 
torch.Size([6])
 
torch.Size([16, 6, 3, 3])
 
torch.Size([16])
 
torch.Size([120, 576])
 
torch.Size([120])
 
torch.Size([84, 120])
 
torch.Size([84])
 
torch.Size([10, 84])
 
torch.Size([10])
 


Let's try a random 32x32 input.
Note: expected input size of this net (LeNet) is 32x32. To use this net on
the MNIST dataset, please resize the images from the dataset to 32x32.



In [ ]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.1104,  0.0295,  0.1502,  0.0137,  0.0178, -0.1712, -0.1503,  0.1474,
         -0.0911,  0.0572]], grad_fn=<AddmmBackward>)


Zero the gradient buffers of all parameters and backprops with random
gradients:



In [ ]:
net.zero_grad()
out.backward(torch.randn(1, 10))

<div class="alert alert-info"><h4>Note</h4><p>``torch.nn`` only supports mini-batches. The entire ``torch.nn``
    package only supports inputs that are a mini-batch of samples, and not
    a single sample.

    For example, ``nn.Conv2d`` will take in a 4D Tensor of
    ``nSamples x nChannels x Height x Width``.

    If you have a single sample, just use ``input.unsqueeze(0)`` to add
    a fake batch dimension.</p></div>

Before proceeding further, let's recap all the classes you’ve seen so far.

**Recap:**
  -  ``torch.Tensor`` - A *multi-dimensional array* with support for autograd
     operations like ``backward()``. Also *holds the gradient* w.r.t. the
     tensor.
  -  ``nn.Module`` - Neural network module. *Convenient way of
     encapsulating parameters*, with helpers for moving them to GPU,
     exporting, loading, etc.
  -  ``nn.Parameter`` - A kind of Tensor, that is *automatically
     registered as a parameter when assigned as an attribute to a*
     ``Module``.
  -  ``autograd.Function`` - Implements *forward and backward definitions
     of an autograd operation*. Every ``Tensor`` operation creates at
     least a single ``Function`` node that connects to functions that
     created a ``Tensor`` and *encodes its history*.

**At this point, we covered:**
  -  Defining a neural network
  -  Processing inputs and calling backward

**Still Left:**
  -  Computing the loss
  -  Updating the weights of the network

Loss Function
-------------
A loss function takes the (output, target) pair of inputs, and computes a
value that estimates how far away the output is from the target.

There are several different
`loss functions <https://pytorch.org/docs/nn.html#loss-functions>`_ under the
nn package .
A simple loss is: ``nn.MSELoss`` which computes the mean-squared error
between the input and the target.

For example:



In [ ]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.7284, grad_fn=<MseLossBackward>)


Now, if you follow ``loss`` in the backward direction, using its
``.grad_fn`` attribute, you will see a graph of computations that looks
like this:

::

    input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
          -> view -> linear -> relu -> linear -> relu -> linear
          -> MSELoss
          -> loss

So, when we call ``loss.backward()``, the whole graph is differentiated
w.r.t. the loss, and all Tensors in the graph that has ``requires_grad=True``
will have their ``.grad`` Tensor accumulated with the gradient.

For illustration, let us follow a few steps backward:



In [ ]:
printatt(loss)

data: 0.7283908724784851
requires_grad: True
grad: None
grad_fn: <MseLossBackward object at 0x7f114db98b38>
is_leaf: False



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

Backprop
--------
To backpropagate the error all we have to do is to ``loss.backward()``.
You need to clear the existing gradients though, else gradients will be
accumulated to existing gradients.


Now we shall call ``loss.backward()``, and have a look at conv1's bias
gradients before and after the backward.



In [ ]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0082,  0.0018,  0.0057, -0.0208, -0.0037, -0.0040])


pay attetion, only leaf node could get grad values, so let's check if they are all leaf

In [ ]:
print_isleaf(net.conv1.weight)
print_isleaf(net.conv1.bias)
print_isleaf(net.conv2.weight)
print_isleaf(net.conv2.bias)
print_isleaf(net.fc1.weight)
print_isleaf(net.fc1.bias)
print_isleaf(net.fc2.weight)
print_isleaf(net.fc2.bias)
print_isleaf(net.fc3.weight)
print_isleaf(net.fc3.bias)

is_leaf: True

is_leaf: True

is_leaf: True

is_leaf: True

is_leaf: True

is_leaf: True

is_leaf: True

is_leaf: True

is_leaf: True

is_leaf: True



Now, we have seen how to use loss functions.

**Read Later:**

  The neural network package contains various modules and loss functions
  that form the building blocks of deep neural networks. A full list with
  documentation is `here <https://pytorch.org/docs/nn>`_.

**The only thing left to learn is:**

  - Updating the weights of the network

Update the weights
------------------
The simplest update rule used in practice is the Stochastic Gradient
Descent (SGD):

     ``weight = weight - learning_rate * gradient``

We can implement this using simple Python code:

.. code:: python

    learning_rate = 0.01
    for f in net.parameters():
        f.data.sub_(f.grad.data * learning_rate)

However, as you use neural networks, you want to use various different
update rules such as SGD, Nesterov-SGD, Adam, RMSProp, etc.
To enable this, we built a small package: ``torch.optim`` that
implements all these methods. Using it is very simple:



In [ ]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

.. Note::

      Observe how gradient buffers had to be manually set to zero using
      ``optimizer.zero_grad()``. This is because gradients are accumulated
      as explained in the `Backprop`_ section.

